# Amazing RL Agent

In [37]:
import gymnasium as gym
import torch
import importlib

# Render screen, set to false if training
RENDER_AGENT = True


In [38]:
# Test pytorch is working

x = torch.rand(size=(1000,))
x = x + 1

# If you have a nvidia gpu with torch installed correctly
# you can move the tensors to the gpu for some extra
# speed, not necessary although.
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Using {DEVICE}")
# When creating a new tensor make sure you move it to a
# device using .to(DEVICE), such as:
x = x.to(DEVICE)
# The tensor in x has been moved to the device


Using cpu


In [39]:
# Import agent
import agents

# Reload module as jupyter notebooks wont update to any changes youve made
importlib.reload(
    agents
)
agent = agents.Agent()


In [40]:
env = gym.make("BipedalWalker-v3", hardcore=True, render_mode="human")
observation, info = env.reset(seed=27)
for _ in range(100):
    action = agent.choose_action(
        env.action_space, observation
    )  # this is where you would insert your policy
    observation, reward, terminated, truncated, info = env.step(action)

    agent.update(observation, reward, terminated, truncated)
    if terminated or truncated:
        observation, info = env.reset()
    env.render()
env.close()
